In [1]:
import numpy as np
#from lhereader import LHEReader

from LHCO_reader import LHCO_reader

import math
import os

# Plotting library
import matplotlib.pyplot as plt

import pandas as pd

In [25]:
!ls ~/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/

bin	     index.html       py.py		  Source
Cards	     lib	      README		  SubProcesses
crossx.html  madevent.tar.gz  README.systematics  TemplateVersion.txt
Events	     MGMEVersion.txt  run_04.lhco
HTML	     myprocid	      RunWeb


# Backgrounds

## p p > W $\gamma$ > $\nu$ + l + $\gamma$ (NOT IMPLEMENTED)

In [ ]:
#Folder = '~/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/'
#datFolder = '/media/andres/disco1tera/ubuntu/Trabajo/CompuTools/la-plata/Wino-radiative-decay/DATA/Wgamma/'

Wgamma_cross = 20.34
Wgamma_runs  = 10
Wgamma_NLO_A = (7.124*10**1) / (2.954*10**1) #W+gamma
Wgamma_NLO_B = (3.713*10**1) / (2.546*10**1) #W+gamma+jet
Wgamma_NLO   = (Wgamma_NLO_A * (7.124/(7.124+3.713)) ) + (Wgamma_NLO_B * (3.713/(7.124+3.713)) )

Wgamma_init_ev = 0
Wgamma_cut_ev  = 0

Wgamma_photon  = []
Wgamma_jet     = []
Wgamma_lepton  = []
Wgamma_MET     = []
Wgamma_hadinfo = []



for ii in range(1, Wgamma_runs + 1):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(ii)+".lhco")
    
    Wgamma_init_ev += len(inputevents)
    
    for i in range(len(inputevents)):

        num_jets = 0
#        num_btag = 0
        HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets

        
        # at least one: photon, jet and lepton (electron or muon)
        if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
            
            if inputevents[i]["jet"][0]["PT"] > 100 and -5.0 < inputevents[i]["jet"][0]["eta"] < 5.0:
                
                if inputevents[i]["photon"][0]["PT"] > 10 and -2.5 < inputevents[i]["photon"][0]["eta"] < 2.5:
                    
                    # how many jets (total) and total hadronic pT
                    for j in range(inputevents[i].number()["jet"]):
                        if inputevents[i]["jet"][j]["PT"] > 20 and -5.0 < inputevents[i]["jet"][j]["eta"] < 5.0:
                            num_jets += 1
                            HT_had += inputevents[i]["jet"][j]["PT"]
                            
                    # save data        
#                     Wgamma_photon.append( [inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]] )
#                     Wgamma_jet.append( [inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]] )
#                     if inputevents[i].number()["electron"] > 0:
#                         Wgamma_lepton.append( [inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]] )
#                     else:
#                         Wgamma_lepton.append( [inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]] )
#                     Wgamma_MET.append( [inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]] )
#                     Wgamma_hadinfo.append( [HT, num_jets] )
                    
                            
                            
                    # photon info:
                    with open(datFolder+'photon.dat', 'a') as f:
                        f.write('%f %f %f\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]))
                        
                    # jet info:
                    with open(datFolder+'jet.dat', 'a') as f:
                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))
                                
                    # lepton info
                    if inputevents[i].number()["electron"] > 0:
                        with open(datFolder+'lepton.dat', 'a') as f:
                            f.write('%f %f %f\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]))
                    else:
                        with open(datFolder+'lepton.dat', 'a') as f:
                            f.write('%f %f %f\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]))
                            
                    # MET (eta=0 by def, but I save phi because I have many jets)
                    with open(datFolder+'MET.dat', 'a') as f:
                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                                 
                    # hadronic info
                    with open(datFolder+'hadinfo.dat', 'a') as f:
                        f.write('%f %d\n' % (HT_had, num_jets))
                    
                    # total events that passed the cuts
                    Wgamma_cut_ev += 1



print('Num initial events: ', Wgamma_init_ev)
print('Total pasaron los cortes: ', Wgamma_cut_ev)

## p p > W jets > $\nu$ l jets

In [48]:
ph_number = []
jet_number = []
e_number = []
mu_number = []

for i in range(len(inputevents)):
    ph_number.append(inputevents[i].number()['photon'])
    jet_number.append(inputevents[i].number()['jet'])
    e_number.append(inputevents[i].number()['electron'])
    mu_number.append(inputevents[i].number()['muon'])
    
    
print('events with photons ', len(np.where(np.asarray(ph_number) > 0)[0]))
print('events with jets ', len(np.where(np.asarray(jet_number) > 0)[0]))
print('events with electrons ', len(np.where(np.asarray(e_number) > 0)[0]))
print('events with muons ', len(np.where(np.asarray(mu_number) > 0)[0]))

('events with photons ', 32558)
('events with jets ', 544922)
('events with electrons ', 155780)
('events with muons ', 206259)


In [29]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/'
datFolder = '../data/Wjets/'

Wjets_cross = 387.0
Wjets_runs  = 4
Wjets_NLO_A = (2.843*10**4) / (2.045*10**4) # W+jet
Wjets_NLO_B = (7.786*10**3) / (6.805*10**3) # W+jet+jet
Wjets_NLO   = (Wjets_NLO_A * (28.43/(28.43+7.786)) ) + (Wjets_NLO_B * (7.786/(28.43+7.786)) )

Wjets_init_ev = 0
Wjets_cut_ev  = 0

Wjets_photon  = []
Wjets_jet     = []
Wjets_lepton  = []
Wjets_MET     = []
Wjets_hadinfo = []



for ii in range(4, Wjets_runs + 1):
    
    inputevents = LHCO_reader.Events(f_name = Folder + "run_0" + str(ii) + ".lhco")
    
    Wjets_init_ev += len(inputevents)
    
    for i in range(len(inputevents)):

        num_jets = 0
#        num_btag = 0
        HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets

        
        # at least one: photon, jet and lepton (electron or muon)
        if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
            
            if inputevents[i]["jet"][0]["PT"] > 100 and -5.0 < inputevents[i]["jet"][0]["eta"] < 5.0:
                
                if inputevents[i]["photon"][0]["PT"] > 10 and -2.5 < inputevents[i]["photon"][0]["eta"] < 2.5:
                    
                    # how many jets (total) and total hadronic pT
                    for j in range(inputevents[i].number()["jet"]):
                        if inputevents[i]["jet"][j]["PT"] > 20 and -5.0 < inputevents[i]["jet"][j]["eta"] < 5.0:
                            num_jets += 1
                            HT_had += inputevents[i]["jet"][j]["PT"]
                            
                    # save data        
                    Wjets_photon.append( [inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]] )
                    Wjets_jet.append( [inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]] )
                    if inputevents[i].number()["electron"] > 0:
                        Wjets_lepton.append( [inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]] )
                    else:
                        Wjets_lepton.append( [inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]] )
                    Wjets_MET.append( [inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]] )
                    Wjets_hadinfo.append( [HT_had, num_jets] )
                    
                            
                            
                    # photon info:
#                   with open(datFolder + 'photon.dat', 'a') as f:
#                       f.write('%f %f %f\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]))
#                        
#                   # jet info:
#                   with open(datFolder + 'jet.dat', 'a') as f:
#                       f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))
#                                
#                   # lepton info
#                   if inputevents[i].number()["electron"] > 0:
#                       with open(datFolder + 'lepton.dat', 'a') as f:
#                           f.write('%f %f %f\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]))
#                   else:
#                       with open(datFolder + 'lepton.dat', 'a') as f:
#                           f.write('%f %f %f\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]))
#                            
#                   # MET (eta=0 by def, but I save phi because I have many jets)
#                   with open(datFolder + 'MET.dat', 'a') as f:
#                       f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
#                                 
#                   # hadronic info
#                   with open(datFolder + 'hadinfo.dat', 'a') as f:
#                       f.write('%f %d\n' % (HT_had, num_jets))
#                    
#                   # total events that passed the cuts
#                   Wjets_cut_ev += 1

print('Num initial events: ', Wjets_init_ev)
print('Total pasaron los cortes: ', Wjets_cut_ev)

('Num initial events: ', 546574)
('Total pasaron los cortes: ', 0)
